In [7]:
import pandas as pd
import numpy as np
import json
import requests
import settings

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

In [8]:
token = 'Bearer ' + settings.access_token
base_url = "http://api.genius.com"
headers = {'Authorization': token}

In [9]:
def getalbumlyrics(urllinks, df_names, album_names):
    for i, albumurl in enumerate(urllinks):
        res = requests.get(albumurl)
        soup = BeautifulSoup(res.content, 'html.parser')
        track_list = soup.find_all('div', {'class' : 'chart_row-content'})
        songs = []
        for track in track_list:
            album = {}
            song = track.find('h3').text.strip().split('\n').pop(0)
            album['song']= song.replace('\xa0', '')
            songs.append(album)
            album['link'] = track.find('a').attrs['href']
            
            globals()[df_names[i]] = pd.DataFrame(songs)
            
            album_lyrics = []
            #song lyrics
            def get_url(url):
                song_lyrics = {}
                page = requests.get(url)
                html = BeautifulSoup(page.text, 'html.parser')
                [h.extract() for h in html('script')]
                lyrics = html.find('div', class_ = 'lyrics').get_text().strip().split("\n")
                
                song_lyrics['lyrics'] = lyrics
                album_lyrics.append(song_lyrics)
                
                globals()[album_names[i]] = pd.DataFrame(album_lyrics)
                
            #grab song lyric links - nested function
            for links in globals()[df_names[i]]['link']:
                get_url(links)

In [10]:
getalbumlyrics(['https://genius.com/albums/Post-malone/Hollywoods-bleeding',
               'https://genius.com/albums/Billie-eilish/When-we-all-fall-asleep-where-do-we-go',
               'https://genius.com/albums/Ariana-grande/Thank-u-next',
               'https://genius.com/albums/Drake/Care-package'],
              ['post', 'billie', 'ariana', 'drake'],
              ['postalbum', 'billiealbum', 'arianaalbum', 'drakealbum'])

In [11]:
post2019 = pd.concat([post, postalbum], axis = 1)
billie2019 = pd.concat([billie, billiealbum], axis = 1)
ariana2019 = pd.concat([ariana, arianaalbum], axis = 1)
drake2019 = pd.concat([drake, drakealbum], axis = 1)


post2019.to_csv('data/post2019.csv', index = False)
billie2019.to_csv('data/billie2019.csv', index = False)
ariana2019.to_csv('data/ariana2019.csv', index = False)
drake2019.to_csv('data/drake2019.csv', index= False)

In [12]:
billie2019.head()

,song,link,lyrics
0,!!!!!!!,https://genius.com/Billie-eilish--lyrics,"[[Spoken: Billie Eilish & FINNEAS], My Invisal..."
1,​bad guy,https://genius.com/Billie-eilish-bad-guy-lyrics,"[[Verse 1], White shirt now red, my bloody nos..."
2,​xanny,https://genius.com/Billie-eilish-xanny-lyrics,"[[Verse 1], What is it about them?, I must be ..."
3,​you should see me in a crown,https://genius.com/Billie-eilish-you-should-se...,"[[Verse 1], Bite my tongue, bide my time, Wear..."
4,​all the good girls go to hell,https://genius.com/Billie-eilish-all-the-good-...,"[[Intro], My Lucifer is lonely, , [Verse 1], S..."
